In [1]:
import pandas as pd
import numpy as np
import re
olympic_data=pd.read_csv('athlete_events.csv')

In [2]:
#Number of countries and athletes participating is inconsistent before 1996 - use only data post 1996
condition_year_1996=olympic_data["Year"]>=1996
olympic_data=olympic_data[condition_year_1996]
condition_summer=olympic_data["Season"]=="Summer"
olympic_data=olympic_data[condition_summer]

In [3]:
# Create a mapping dictionary for corrections
# Replace incorrect names with correct ones

corrections = {
    "Athina": "Athens",
    "Roma": "Rome",
    "Moskva": "Moscow",
    "Sankt Moritz": "St. Moritz"
}


olympic_data['City'] = olympic_data['City'].replace(corrections)

In [7]:
#Investigating quality of ID column
#Duplicates exist but this is intentional, as one athlete can participate in more than one event/olympics.
#ID numbering is consistent throughout years - e.g. same ID for athlete in different olympic games.
#ID is based on alphabetical order of last name
#no missing values in the ID column
#Team column comments: Team names are inconsistent - mixed with countries and sports team names. We will use NOC column instead for our analysis

In [9]:
#Replace Na values as "No Medal" string as null values correspond to no winning athletes.
olympic_data["Medal"] = olympic_data["Medal"].apply(lambda x:"No Medal" if pd.isna(x) else x)

In [11]:
#The difference between the medal count can be due to differences on the number of team sport members and ties.
olympic_data["Medal"].value_counts()

Medal
No Medal    69395
Bronze       4078
Gold         3903
Silver       3878
Name: count, dtype: int64

In [13]:
olympic_data.drop_duplicates(inplace = True)
print(olympic_data.duplicated().sum())

0


In [15]:
#Deduplicating the number of medals from team sports
#Creat a dataframe just with the result of each olympic by event (Gold, Silver and Bronze)
olympic_data["Concat"]=olympic_data["Games"] + olympic_data["Event"] +olympic_data["Medal"]+olympic_data["NOC"]
olympic_data_podium=olympic_data[olympic_data["Medal"]!= "No Medal"]
olympic_data_podium.drop_duplicates(subset=["Concat"], inplace= True)
olympic_data_podium.Concat.duplicated().sum()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_27252\2009509916.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olympic_data_podium.drop_duplicates(subset=["Concat"], inplace= True)


np.int64(0)

In [23]:
#Creat a new file droping teams duplicated
#Create a new dataframe
olympic_data_not_duplicated=olympic_data
olympic_data_not_duplicated["Concat"]=olympic_data_not_duplicated["Games"] + olympic_data_not_duplicated["Event"] +olympic_data_not_duplicated["Medal"]+olympic_data_not_duplicated["NOC"]
olympic_data_not_duplicated.drop_duplicates(subset=["Concat"], inplace= True)
olympic_data_not_duplicated.Concat.duplicated().sum()

#NOC column - we want to merge the NOC column in our Olympics dataset with the region column from the NOC regions dataset
noc_regions = pd.read_csv('noc_regions.csv')

#Merge with NOC
olympic_data_not_duplicated= pd.merge(
    olympic_data_not_duplicated, 
    noc_regions[['NOC', 'region']], 
    on='NOC', 
    how='left')

In [27]:
# Save the updated DataFrame to a new CSV file
olympic_data_not_duplicated.to_csv('teams_not_duplicated_summer_olympics_1996-2016_deduplicate_team_medals.csv', index=False)

In [ ]:
#NOC column - we want to merge the NOC column in our Olympics dataset with the region column from the NOC regions dataset
noc_regions = pd.read_csv('noc_regions.csv')

#Merge the datasets on the 'NOC' column
olympics_updated = pd.merge(
    olympic_data_podium, 
    noc_regions[['NOC', 'region']], 
    on='NOC', 
    how='left'
)

In [ ]:
# Save the updated DataFrame to a new CSV file
olympics_updated.to_csv('summer_olympics_1996-2016_deduplicate_team_medals.csv', index=False)

In [ ]:
#import pandas as pd
#import numpy as np
#import re
#olympic_data=pd.read_csv('MAIN_summer_olympics_1996-2016_deduplicate_team_medals.csv')
#olympic_data.region.fillna("Singapore",inplace=True)
#olympic_data.region.isnull().sum()
#olympic_data_hour=olympic_data[olympic_data.region !="Kosovo"]
#olympic_data_hour

In [ ]:
#time_zones_athlete=pd.read_csv("time_zones2.csv", encoding="latin-1", sep=";")
#time_zones_athlete

In [ ]:
#time_zones_athlete[["Hours","Minutes"]]=time_zones_athlete["GMT offset"].str.split(":", expand=True)
#time_zones_athlete
#time_zones_athlete.Minutes.fillna("00",inplace=True)
#time_zones_athlete.head(40)

#time_zones_athlete["Minutes"]=time_zones_athlete["Minutes"].map({
    #"30": 0.5,
    #"00": 0,
    #"0": 0
#})
#time_zones_athlete["Hours"]=time_zones_athlete["Hours"].astype(float).abs()
#time_zones_athlete["GMT Athlete Country"]=time_zones_athlete["Hours"]+time_zones_athlete["Minutes"]

#time_zones_athlete

In [ ]:
#time_zones_athlete.rename(columns={"Country":'region'}, inplace=True)
#time_zones_athlete

In [ ]:
#olympic_data.columns

In [ ]:
#olympic_data_hour=pd.merge(
    #olympic_data, 
    #time_zones_athlete[["region","GMT Athlete Country","Capital city"]],
    #on="region",
    #how="left")

#olympic_data_hour

In [ ]:
#print(olympic_data_hour.region.isnull().sum())
#print(olympic_data_hour["GMT Athlete Country"].isnull().sum())
#print(olympic_data_hour["Capital city"].isnull().sum())

In [ ]:
#olympic_data_hour[olympic_data_hour["GMT Athlete Country"].isnull()].head()

In [ ]:
#olympic_data_hour.dropna(subset=["GMT Athlete Country"], inplace=True)
#print(olympic_data_hour.region.isnull().sum())
#print(olympic_data_hour["GMT Athlete Country"].isnull().sum())
#print(olympic_data_hour["Capital city"].isnull().sum())

In [ ]:
#time_zones_host=time_zones_athlete
#time_zones_host.rename(columns={"Capital city":'City'}, inplace=True)
#time_zones_host.rename(columns={"GMT Athlete Country":'GMT Host City'}, inplace=True)
#time_zones_host

In [ ]:
#olympic_data_hour=pd.merge(
    #olympic_data_hour, 
    #time_zones_host[["City","GMT Host City",]],
    #on="City",
    #how="left")

#olympic_data_hour

In [ ]:
#olympic_data_hour.isnull().any()

In [ ]:
#olympic_data_hour.info()

In [ ]:
#olympic_data_hour.info()

In [ ]:
#olympic_data_hour["Time difference"]=(olympic_data_hour["GMT Host City"]-olympic_data_hour["GMT Athlete Country"]).abs()
#olympic_data_hour

In [ ]:
#olympic_data_hour.groupby("Time difference")["Medal"].count()


In [ ]:
#def create_range_time(time):
    #if time <= 2.0:
        #return "[0-2.0]"
    #elif time <=4:
        #return "(2.0-4.0]"
    #elif time <=6:
        #return "(4.0-6.0]"
    #elif time <=8:
        #return "(6.0-8.0]"
    #elif time <=10:
        #return "(8.0-10.0]"
    #elif time <=12:
        #return "(10.0-12.0]"
    #else:
        #return "(12.0-13.00]"

In [ ]:
#olympic_data_hour["Range Time"]=olympic_data_hour["Time difference"].apply(create_range_time)
#olympic_data_hour

In [ ]:
#olympic_data_hour.groupby("Range Time")["Medal"].count().sort_values(ascending=False)